In [ ]:
##################################################
##### author: Debajyoti Dutta
##### email: debajyoti.dutta@dxc.com
##### last_update_date: 2020-12-10
##### version: 2-dot-1
##################################################
import os
import sys
import pandas as pd
import numpy as np
import csv
from pprint import pprint
import datetime
from datetime import datetime
from pandas import ExcelWriter
import re
import string

In [ ]:
### GLOBAL CONFIGURATIONS
source_column_names = ["Assettag", "Serial", "Systemname", "NetID", "Asset_Number", "Registry", "Number_2", "SoftwareTitle", "SoftwareVersion", "ScanDate", "OSVersion", "OSEdition", "Number_3"]
dump_source_location =""
report_store_name=""
### ALL FUNCTION DEFINITIONS
##############################################################################################################################
##### ##### START DEFINITION OF -- setting_locations()
def setting_locations():
    global dump_source_location
    dump_source_location=input(r'Please mention the full path of report dump source folder :: ')  
    #dump_source_location =r"C:\DEB_CloudData\CBA_BAM"
    global report_store_name
    report_store_name = "Custom_Reports_"
    if os.path.exists(os.path.join(dump_source_location, report_store_name)) \
                      and not os.path.isfile(os.path.join(dump_source_location, report_store_name)):
#         print(" .. Custom Reports store exists ...\n")
        os.chdir(os.path.join(dump_source_location, report_store_name))
    else:
        os.mkdir(report_store_name)
#         print(" Custom Reports store created\n")
#         print("-"*30)
#         os.chdir(os.path.join(dump_source_location, report_store_name))

    return dump_source_location
##### ##### ##### END DEFINITION OF -- setting_locations()
##############################################################################################################################
##### ##### START DEFINITION OF -- source_report_selection()
def source_report_selection(dump_source_location_):
    dump_source_folder = os.listdir(dump_source_location_)
    all_files_dict = {} ## define the dictionary for the folder contents
    selected_files_list = [] ## define the list to capture the files chosen by user
    decision="" ## looping variable for raw data source file selection
    while decision != 'x':
        print("SELECT THE RAW DATA FILES ...\n (CHOOSE THE CORRESPONDING FILE NUMBER/S) \n\t ( for MULTIPLE FILES please use Comma \",\" in between ) \n TO Select All press - A/a \n TO EXIT press - X/x)" )
        print("-"*40)      
        i = 1
        for file_ in dump_source_folder: ## capture the folder contents as a dictionary                                                                                             #
            all_files_dict[i] = file_
            i +=1 
        pprint(all_files_dict) ## showing the file list to the user to choose the raw data file to scan
        print("-"*30)
    #     selected_files=input(': ')
    #     Selected_Files_=selected_files.split(sep= ",")
        Selected_Files_=input('::  ').split(sep= ",")## capturing the user's choice - input string >> split to >> list 
        if Selected_Files_[0] == "a" or Selected_Files_[0] == "A":
            count = 1
            for i in dump_source_folder:
                print("{}. {}".format(count, i))
                selected_files_list.append(i)
                count += 1
        elif Selected_Files_[0] == "x" or Selected_Files_[0] == "X":
            return
        else:
            print("\nYou have selected the following reports : ", end = "")
    #     print(Selected_Files_) ## shows the chosen file numbers to user
            print("\n")
            for i in Selected_Files_:
                if i != " " and i != '':
                    print("\t", end = "")
                    print(i, end = " : ",)
                    i_=int(i)
                    print(all_files_dict[i_]) ## display the values picked from the dict
                    selected_files_list.append(all_files_dict[i_]) ## populate the list from the data picked from the dict
        print("\nProceed to Scan the reports by  (Y/y) or \n\t Change report selection by  (N/n) OR \n\t Exit the script by  X/x ")
        ## capture user's decision and proceed accordingly
        decision=input(r'::  ').lower()  
        if decision == 'y':
#             print(Selected_Files_, selected_files_list)
            return selected_files_list
        elif decision == 'n':
            os.system('CLS')
            print("-"*30)
            print("-"*10, end =" ")
            print(".. please select AGAIN ...\n")
            selected_files_list.clear()
        elif decision == 'x':
            print("Exiting")
            sys.exit("Exiting")
            selected_files_list.clear()
        else:
            print("Please select appropriately\n")
            selected_files_list.clear()
##### ##### ##### END DEFINITION OF -- source_report_selection()
##############################################################################################################################
##### ##### START DEFINITION OF -- dump_data_scanning()
def dump_data_scanning(selected_source_reports_, dump_source_location_, source_column_names_):
    os.chdir(dump_source_location_)
    collated_df = pd.DataFrame()
    print("-"*60)
    print("\n...Raw Data scan started..\n")
    for file_ in selected_source_reports_:
        print(file_, end = " ...processing... ")
        temp_df = pd.DataFrame()
        with open(file_) as csvfile_:
            dialect=csv.Sniffer().sniff(csvfile_.read(14734))
        csv_in_chunks=pd.read_csv(file_, header=None, iterator=True, chunksize=100000, sep=",", quotechar='"', names=source_column_names_, encoding="ISO-8859-1", index_col=None, engine='python', error_bad_lines=False, dialect=dialect)
        temp_df=pd.concat(csv_in_chunks, ignore_index=True)
        ## concating all rows in the dataframe 
        rows_, cols_ = temp_df.shape
        print(" File has {} rows and {} columns.. PLEASE HOLD ...".format(rows_, cols_))
        temp_df["Filename"]=file_
#         print(temp_df.shape)
        collated_df=pd.concat([collated_df, temp_df], ignore_index=True)
        ## collating all the dataframes
    print("\n...Raw Data scan completed..\n")
    print("-"*60)
    print("--- GENERATING PIVOTED SUMMARY FOR COLUMN 'SoftwareTitle' ----- please hold -----")
    os.chdir(os.path.join(dump_source_location, report_store_name))
#     collated_df.columns = collated_df.columns.get_level_values(0)
    collated_df_pivoted1 = pd.pivot_table(collated_df, values='Systemname', index=['SoftwareTitle'], columns=['OSVersion'], aggfunc=pd.Series.count, margins=True)
    collated_df_pivoted2 = pd.pivot_table(collated_df, values='Systemname', index=['SoftwareTitle','OSVersion'], aggfunc=pd.Series.count, margins=True)
    collated_df_grouped = collated_df.groupby(by=["SoftwareTitle","OSVersion"]).count()
#     collated_df_pivoted = pd.pivot_table(collated_df, values='Systemname', index=['SoftwareTitle'], columns=['OSVersion'], aggfunc='count') # old style count
#     print(collated_df.index, collated_df.columns, collated_df_pivoted, collated_df_grouped["Assettag"])
    file_ = "Pivot_Result_" + datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + ".xlsx"
#     print(file_)
    with pd.ExcelWriter(file_) as writer:
        collated_df_pivoted1.to_excel(writer, sheet_name="Pivot_Type_1", engine='xlsxwriter', header=True)
        collated_df_pivoted2.to_excel(writer, sheet_name="Pivot_Type_2", engine='xlsxwriter', header=True)
        collated_df_grouped.to_excel(writer, sheet_name="GroupBy_Result", engine='xlsxwriter', header=True)
        print("\nSAVED TO FILE...  "+ file_ + "  in  " + os.getcwd()+"\n")
    writer.save()
    print("-"*40)
    return collated_df
##### ##### ##### END DEFINITION OF -- dump_data_scanning()
##############################################################################################################################
##### ##### START DEFINITION OF -- apply_filter()
def apply_filter(collated_df_, source_column_names_):
    print("\nPlease select the column number of the filter/s you want to use \n\t ( for MULTIPLE FILTERS please use Comma \",\" in between ) \n The script will apply the filters in the sequence you mention here ..")
    print("\n\nThe avaiable report filters are .. \n")
    print("-"*20)
    count=1
    for i in source_column_names_:
        print("{}. {}".format(count, i))
        count += 1
    print("-"*20)
    Selected_filters=input(r'Enter filter number/s ::  ').split(sep=",") ## choosing the column headers to be used for filtering
    print("-"*20)
    chosen_columns = []
    keywords = []
    sp_chars = string.punctuation
    for i in Selected_filters:
        if i != " " and i != '':
            i_=(int(i)- 1)
            print("\nPlease specify the search keyword for column -- \"{}\" \n ( SPECIAL CHARACTERS  ONLY DOT(.) AND SPACE  ARE ALLOWED)\n".format(source_column_names_[i_]))
            ## checking for special characters
            SPChar_check = True
            while SPChar_check != False:
                keyword=input(r'::  ')
                keyword_split = [keyword[i:i+1] for i in range(0, len(keyword), 1)]
                bool_List_For_SP_chars = list(map(lambda char: char in sp_chars, keyword_split))
                SPChar_check = False
                for (input_chars, bool_idx) in zip(keyword_split, bool_List_For_SP_chars):
                    if bool_idx == True and input_chars != ".":
                        print("unaccepted character - \"{}\" used in the search keyword ".format(input_chars))
                        SPChar_check = True
                if SPChar_check == True:
                    print("Please try again ...")
            chosen_columns.append(source_column_names_[i_])
            keywords.append(keyword)
#     print(chosen_columns)
#     print(keywords)
    filtered_df = collated_df
    for (col,kw) in zip(chosen_columns, keywords):
#         print(i,j)
#         filtered_df = filtered_df[filtered_df[col].str.contains(kw, case=False, na=False) ==True]
        filtered_df = filtered_df[filtered_df[col].str.contains(pat = kw, case=False, na=False,regex=True) ==True]
#         filtered_df = filtered_df[filtered_df[col].str.extractall(kw) ==True]
    print("...s e a r c h i n g  f o r  ......", end ="  ")
    print(keywords)
#     print(filtered_df.shape)
    if filtered_df.shape[0] == 0:
        print("\nNo matching records found... ")
    else:
        print("\nSearch returned {} records in...".format(filtered_df.shape[0]))
        filtered_df_grouped=filtered_df.groupby(by=["Filename"]).count()
        print(filtered_df_grouped["Systemname"])
        print("-"*20)
    next_step = ""
    while next_step != 'x':
        print("\nView seacrh results sample on screen (W/w) \
          \nGenerate the search report as Excel file (R/r) \
          \nTry a new Search (N/n) \
          \nExit the script (X/x)")
        next_step=input(r'::  ').lower()
        if next_step == 'w':
            print(filtered_df.head())
            print("-"*40)
            carry_on_= input(r'Save this search result in a file (V/v) OR Skip (K/k) ::  ').lower()
            if carry_on_ == 'v':
                save_search_to_excel(filtered_df, chosen_columns, keywords)
            elif carry_on_ == 'k':
                break
            else:
                print("\nPlease select appropriately\n")
        elif next_step == 'n':
            print("-"*30)
            print("-"*30)
            apply_filter(collated_df, source_column_names)
        elif next_step == 'r':
            save_search_to_excel(filtered_df, chosen_columns, keywords)
            print("-"*40)
            carry_on_= input(r'Start a new Search (S/s) OR Exit (X/x) ::  ').lower()
            if carry_on_ == 's':
                apply_filter(collated_df, source_column_names)
            elif carry_on_ == 'x':
                print("Exiting")
                sys.exit("Exiting")
            else:
                print("\nPlease select appropriately\n")
        elif next_step == 'x':
            print("Exiting")
            sys.exit("Exiting")
        else:
            print("\nPlease select appropriately\n")
##### ##### ##### END DEFINITION OF -- apply_filter()
##############################################################################################################################
##### ##### START DEFINITION OF -- save_search_to_excel()
def save_search_to_excel(filtered_df_, chosen_columns_, keywords_):
# def save_report_to_excel():
    global report_store_name
    print("-"*40)
    os.chdir(dump_source_location)
    os.chdir(report_store_name)  
    timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    report_folder_name = timestamp
    os.mkdir(report_folder_name)
    os.chdir(os.path.join(dump_source_location, report_store_name, report_folder_name))
    ## excel filename creation
    name_ = ""
    for i in keywords_: ## for single or multi keyword cases
        name_ += i + "-" 
    name_="Results-for__"+name_+".xlsx" 
    ## actual excel writing begins here
    total_rows, total_columns = filtered_df_.shape
    max_excelrows_per_worksheet = (1048576 - 1) ## total number of  rows allowed per worksheet minus 1 for header row
    start_pos = 0 ## represents first row of the data frame
    end_pos = (max_excelrows_per_worksheet - 1) ## since the start row of dataframe has index 0 instead of index 1
    sheet_count=0
    if total_rows > max_excelrows_per_worksheet: ## case for multisheet writing ... a single excel worksheet supports only 10,48,576 rows.. skipping the first row for column header
        print("... WRITING EXCEL FILE - PLEASE HOLD ...")
        with pd.ExcelWriter(name_) as writer:
            while True :
#                 print("start_pos {}, end_pos {}".format(start_pos, end_pos))
                if end_pos < total_rows :
                    df_chunk = filtered_df_.iloc[start_pos:end_pos,:]
                    sheet_count += 1    
                    df_chunk.to_excel(writer, sheet_name=f'Sheet {sheet_count}', engine='xlsxwriter', header=True, index=False)
                    writer.save()
                    print("Added UPTO {}th record from search results in Sheet num {}".format(end_pos+1,sheet_count))
                    start_pos = (end_pos + 1)
                    end_pos = (end_pos + max_excelrows_per_worksheet)          
                else:
                    df_chunk = filtered_df_.iloc[start_pos:,:]
                    sheet_count += 1
                    df_chunk.to_excel(writer, sheet_name=f'Sheet {sheet_count}', engine='xlsxwriter', header=True, index=False)
                    writer.save()
                    print("Added the LAST {} records from search results in Sheet num {}".format(total_rows-start_pos,sheet_count))
                    break
    else:
        print("... WRITING EXCEL FILE - PLEASE HOLD ...", end=" ") ## case for single sheet writing - total_rows <= max_excelrows_per_worksheet
        with pd.ExcelWriter(name_) as writer:
            sheet_count += 1 
            filtered_df_.to_excel(writer, sheet_name=f'Sheet {sheet_count}', engine='xlsxwriter', header=True, index=False)
            print("Added {} records from search results in Sheet num {}".format(total_rows,sheet_count))
            writer.save()
    print("\n\nReport saved in file \"{}\" ".format(name_))
    print("\nTo access the above report please visit >> {}\n".format(os.getcwd()))
##### ##### ##### END DEFINITION OF -- save_search_to_excel()

In [ ]:
##### os.system('CLS')
print("-"*60)
print("-"*20, end=" ")
print("Step 0 - SETTING LOCATIONS")
print("-"*60)
setting_locations()
# print(dump_source_location)
########################################################################################################################
os.system('CLS')
print("-"*60)
print("-"*20, end=" ")
print("Step 1 - SELECTING THE DUMP DATA")
print("-"*60)
os.chdir(dump_source_location)
selected_source_reports = source_report_selection(dump_source_location)
########################################################################################################################
os.system('CLS')
if selected_source_reports == None:
    print("Exiting")
    sys.exit("Exiting")
print("-"*60)
print("-"*20, end=" ")
print("Step 2 - DATA PROCESSING AND SUMMARY")
print("-"*60)
collated_df=dump_data_scanning(selected_source_reports, dump_source_location, source_column_names)
########################################################################################################################
if input(r'-- P R E S S  X/x TO EXIT     O R  A N Y  O T H E R  K E Y  T O  C O N T I N U E  -- ').lower() == "x":
    sys.exit("Exiting")
os.system('CLS')
if collated_df.empty :
    print("Exiting")
    sys.exit("Exiting")
print("-"*60)
print("-"*20, end=" ")
print("Step 3 - DATA FILT7ERING AND CUSTOM REPORTS")
print("-"*60)
apply_filter(collated_df, source_column_names)
########################################################################################################################
#######################################################################################################################